In [1]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import RMSprop, Adam
import tensorflow as tf
import csv 

In [2]:
data = pd.read_csv('./datasets/naver_movie_all.csv')
data = data[['document', 'label', 'star']]
print(data.shape)
data.head()

(40947, 3)


,document,label,star
0,'그래도 우리 잘 해냈지?'라고 말하는 듯한 마지막 눈빛교환이 잊혀지질 않는다,1,9
1,"와..그냥 감탄밖엔 안 나온다. 영상미며 음악이며..조폭, 건달, 검사, 신파 좋아...",1,9
2,처음 들을때는 몰랐다 이 음악이 슬퍼질거란걸,1,7
3,"꿈과 사랑, 우리는 항상 성공인지 실패인지 평가만 해왔다. 하지만 라라랜드는 말해준...",1,10
4,마지막 회상신에서 처음과 달리 키스신으로 스토리시작될때 진짜 소리지를뻔...,1,9


In [3]:
train_data, test_data = train_test_split(data, test_size=0.33, random_state=777)

In [4]:
print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력
print('테스트용 리뷰 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력

훈련용 리뷰 개수 : 27434
테스트용 리뷰 개수 : 13513


In [5]:
train_data[:5] # 상위 5개 출력

,document,label,star
6655,넷플릭스에도 내주세요ㅠㅠㅠㅠ,1,10
2860,세상에... 수퍼맨의 액션을 제대로 보여주다니... 만화에서도 표현하지 못할 영상미...,1,10
21868,저렴하게 만들었네.. 비용 얼마 안들었겠다,0,4
6166,인생을 담은 내 인생영화를 통해 조금이나마 인생을 배웠다,1,10
30524,재난영화인줄 알고 밧다가 2시간 진짜 통으로 잣다 기대말고 봐라 시간아까움,0,2


In [6]:
print('테스트용 리뷰 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력

테스트용 리뷰 개수 : 13513


In [7]:
test_data[:5]

,document,label,star
21182,인생은 이미지다,1,9
33369,무서울지 알았는데 사회적 메세지가 담겨 있어서 슬펐네요ㅠ 어린이들을 지켜주는 어른들...,1,10
37624,기대없이 봤는데 그 이상을 보여준 영화 이 영화를 왜 이제야 본걸까..,1,10
5063,난생처음 평점댓글달아본다..나의 인생영화라해도 과언이아니다..여기적힌댓글들을 다 이...,1,10
5640,여러분 제발 영화 상영하고 있을 때 놓치지말고 영화관 가서 보세요... 제발.. 이...,1,10


### Preprocessing

In [8]:
# 데이터 중복 확인
train_data['document'].nunique(), train_data['label'].nunique()

(24084, 2)

In [9]:
# document 열에서 중복인 내용이 있다면 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True) 

/Users/jshin/opt/anaconda3/envs/ml2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
print('총 샘플의 수 :',len(train_data))

총 샘플의 수 : 24084


In [ ]:
# train_data['label'].value_counts().plot(kind = 'bar')

In [11]:
# 긍정과 부정의 수를 확인한다 (label=0이면 부정, label=1이면 긍정으로 표시한다)

# train_data['label'].value_counts()
print(train_data.groupby('label').size().reset_index(name = 'count'))

   label  count
0      0   5748
1      1  18336


In [12]:
train_data['label'].value_counts()

1    18336
0     5748
Name: label, dtype: int64

In [13]:
# 결측치 확인하기 - 나중참고용
# print(train_data.isnull().sum())
train_data.loc[train_data.document.isnull()]
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거

print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인
print(train_data.isnull().any()) # Null 값이 존재하는지 확인
# print(len(train_data))

False
document    False
label       False
star        False
dtype: bool


In [14]:
# 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

# 알파벳과 공백을 제외하고 모두 제거(나중에 영어텍스트만 남기고 싶을때 사용)
# train_data['document'] = train_data['document'].str.replace("[^a-zA-Z ]","")

In [ ]:
# https://www.unicode.org/charts/PDF/U3130.pdf
# ㄱ ~ ㅎ: 3131 ~ 314E
# ㅏ ~ ㅣ: 314F ~ 3163
# https://www.unicode.org/charts/PDF/UAC00.pdf
# 가 ~ 힣

In [ ]:
train_data[:5]

In [15]:
# 영어로만 작성된 리뷰도 삭제 됨
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

document    0
label       0
star        0
dtype: int64


In [16]:
train_data.loc[train_data.document.isnull()][:5]

,document,label,star


In [17]:
train_data = train_data.dropna(how = 'any')
print(len(train_data))

24084


In [18]:
save_csv = train_data[:100]
save_csv.to_csv('tain_data.csv')

### J 새로추가

In [ ]:
train_data

In [ ]:
### test 데이터 전처리(tain과 동일)

In [20]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

전처리 후 테스트용 샘플의 개수 : 12618


### Tokenization

In [ ]:
# 불용어 제거, 한국어의 조사, 접속사 등 -> 지속적으로 검토하면서 추가해서 삭제
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [21]:
# 토큰화를 위한 형태소 분석기는 KoNLPy의 Okt
okt = Okt()
# okt.morphs('2000년이후 최고의 서스펜스... 라고 했지만, 이걸 왜 봤을까', stem = True)

In [23]:
ko_file = pd.read_csv('ko.csv', index_col=0)
ko_file

""
아
휴
아이구
아이쿠
아이고
어
...
일곱
여덟
아홉


In [ ]:
jdata = train_data

jdata

### 명사만 추출하기

In [24]:
# 불용어 제거
# 형태소 토큰화
X_train = []
for sentence in train_data['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in ko_file] # 불용어 제거
    X_train.append(temp_X)

In [30]:
X_train[10]

['옛날', '영화', '느낌', '핵', '노잼', '뮤지컬', '들어가다', '평점', '보다', '평론가', '들', 'ㅋㅋ']

In [35]:
from collections import Counter

for i in X_train:
    c = Counter(X_train)
    print(c.most_common(10))
    

TypeError: unhashable type: 'list'

In [ ]:
print(X_train[0])

In [ ]:
for i in X_train:
    print(i)



In [ ]:
X_test = []
for sentence in test_data['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in ko_file] # 불용어 제거
    X_test.append(temp_X)

In [ ]:
### 정수 인코딩

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
print(tokenizer.word_index)

In [ ]:
threshold = 2 # 빈도수가 3회 미만은 제거
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

In [ ]:
print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
vocab_size = total_cnt - rare_cnt + 1 # 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거. 0번 패딩 토큰을 고려하여 +1
print('단어 집합의 크기 :',vocab_size)

In [ ]:
import pickle
vocab_size = vocab_size

with open('./save/vocab_size.p', 'wb') as file:    # james.p 파일을 바이너리 쓰기 모드(wb)로 열기
    pickle.dump(vocab_size, file)

In [ ]:
# 토크나이저는 텍스트 시퀀스를 숫자 시퀀스로 변환하는 정수 인코딩 과정에서 
# 이보다 큰 숫자가 부여된 단어들은 아예 계산하지 않음
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_train[:3])

In [ ]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

### 빈 샘플(Empty samples) 제거

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]
drop_test = [index for index, sentence in enumerate(X_test) if len(sentence) < 1]

In [ ]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

In [ ]:
X_test = np.delete(X_test, drop_test, axis=0)
y_test = np.delete(y_test, drop_test, axis=0)
print(len(X_test))
print(len(y_test))

In [ ]:
np.save("./save/y_train.npy", y_train)
np.save("./save/y_test.npy", y_test)

### 패딩

In [ ]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
# 전체 샘플 중 길이가 max_len 이하인 샘플의 비율이 몇 %인지 확인하는 함수
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if(len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [ ]:
max_len = 50
below_threshold_len(max_len, X_train)

In [ ]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
np.save("./save/X_train.npy", X_train)
np.save("./save/X_test.npy", X_test)

## LSTM으로 네이버 영화 리뷰 감성 분류

In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
T = X_train.shape[1]
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
i = Input(shape=(T,))
x = Embedding(vocab_size, 100)(i)
x = LSTM(128)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(i, x)
model.summary()

In [ ]:
X_train.shape[1]

In [ ]:
import os

checkpoint_path = "training_1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   verbose=1,
                   patience=4)

mc = ModelCheckpoint(checkpoint_path,
                     monitor='val_acc',
                     mode='max',
                     verbose=1,
                     save_best_only=True,
                     save_weights_only=True,
                     period=5)


model.save_weights(checkpoint_path.format(epoch=0))
model.compile(optimizer=Adam(lr=0.2), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, 
                    callbacks=[es, mc], batch_size=128, validation_split=0.3, verbose=2)


In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

In [ ]:
! ls {checkpoint_dir}

In [ ]:
## 훈련의 손실치를 그래프에 표시
# loss: 0.0696 - accuracy: 0.9748 - val_loss: 0.4810 - val_accuracy: 0.8722

plt.title('Loss')
plt.plot(history.history['loss'], 'b', marker='.', label='loss')
plt.plot(history.history['val_loss'], 'r', label='val_loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model.load_weights(latest)
loss, acc = model.evaluate(X_test,  y_test, verbose=2)

print("\n테스트 정확도: %.4f" % (loss, acc)[1])

In [ ]:
# loaded_model = load_model(latest)
# print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1], verbose=1))

In [ ]:
# 검증 데이터 손실(val_loss)이 증가하면, 과적합 징후
# 검증 데이터 손실이 4회 증가하면 학습을 조기 종료(Early Stopping)
# 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델을 저장
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
# mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', 
#                      verbose=1, save_best_only=True)

In [ ]:
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# history = model.fit(X_train, y_train, epochs=15, 
#                     callbacks=[es, mc], batch_size=60, validation_split=0.2, verbose=2)

In [ ]:
# loaded_model = load_model('best_model.h5')
# print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))
# # print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))